In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential,Model,initializers,layers,Input
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.model_selection import train_test_split


In [2]:
dir=r'D:\dataset\butterfly\train'
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
dir,
labels="inferred",
label_mode="int",
class_names=None,
color_mode="rgb",
batch_size=4,
image_size=(224, 224),
shuffle=True,
seed=10,
validation_split=0.1,
subset='training',
interpolation="gaussian",
follow_links=False,
crop_to_aspect_ratio=False,)


validation_ds=tf.keras.preprocessing.image_dataset_from_directory(
dir,
labels="inferred",
label_mode="int",
class_names=None,
color_mode="rgb",
batch_size=4,
image_size=(224, 224),
shuffle=True,
seed=10,
validation_split=0.1,
subset='validation',
interpolation="gaussian",
follow_links=False,
crop_to_aspect_ratio=False,)

normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255.)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

Found 1257 files belonging to 10 classes.
Using 1132 files for training.
Found 1257 files belonging to 10 classes.
Using 125 files for validation.


In [3]:
def Stem(input):
    x=layers.Conv2D(filters=64,kernel_size=(7,7),strides=(2,2),padding='same',activation='relu')(input)
    x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
    x=layers.LayerNormalization()(x)
    x=layers.Conv2D(filters=192,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x=layers.Conv2D(filters=192,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x)
    x=layers.LayerNormalization()(x)
    x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)    
    return x


def Inception_Block(x,f1,f2,f3,f4,f5,f6):

    x1=layers.Conv2D(filters=f1,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
          
    x2=layers.Conv2D(filters=f2,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x2=layers.Conv2D(filters=f3,kernel_size=(3,3),strides=(1,1),padding='same',activation='relu')(x2)

    x3=layers.Conv2D(filters=f4 ,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x)
    x3=layers.Conv2D(filters=f5,kernel_size=(5,5),strides=(1,1),padding='same',activation='relu')(x3)
       
    x4=layers.MaxPooling2D(pool_size=(3,3),strides=(1,1),padding='same')(x)
    x4=layers.Conv2D(filters=f6,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(x4)
    
    return tf.concat([x1,x2,x3,x4],axis=3)

def AUX1(input):
    ax=layers.AveragePooling2D(pool_size=(1,1),strides=(1,1))(input)
    ax=layers.Conv2D(filters=128,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(ax)
    ax=layers.Flatten()(ax)
    ax=layers.Dense(units=1024,activation='relu')(ax)
    ax=layers.Dropout(0.7)(ax)
    ax=layers.Dense(units=10,activation='softmax',name='ax1')(ax)
    return ax

def AUX2(input):
    ax=layers.AveragePooling2D(pool_size=(1,1),strides=(1,1))(input)
    ax=layers.Conv2D(filters=128,kernel_size=(1,1),strides=(1,1),padding='same',activation='relu')(ax)
    ax=layers.Flatten()(ax)
    ax=layers.Dense(units=1024,activation='relu')(ax)
    ax=layers.Dropout(0.7)(ax)
    ax=layers.Dense(units=10,activation='softmax',name='ax2')(ax)
    return ax


def Classifier(input):
    fc=layers.AveragePooling2D(pool_size=(7,7),strides=(1,1))(input)
    fc=layers.Flatten()(fc)
    fc=layers.Dense(units=10,activation='softmax',name='output')(fc)    
    return fc

In [4]:
inputs=Input(shape=(224,224,3))
x=Stem(inputs)
x=Inception_Block(x,256,256,256,256,256,256)
x=Inception_Block(x,480,480,480,480,480,480)
x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
x=Inception_Block(x,512,512,512,512,512,512)

ax1=AUX1(x)
x=Inception_Block(x,512,512,512,512,512,512)
x=Inception_Block(x,512,512,512,512,512,512)
x=Inception_Block(x,528,528,528,528,528,528)

ax2=AUX2(x)
x=Inception_Block(x,832,832,832,832,832,832)
x=layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
x=Inception_Block(x,832,832,832,832,832,832)
x=Inception_Block(x,1024,1024,1024,1024,1024,1024)
output=Classifier(x)
   
model = tf.keras.models.Model(inputs,[output,ax1,ax2],name = 'GoogleNet')

In [5]:
lr_rate=tf.keras.optimizers.schedules.ExponentialDecay(1e-3, 10000, 0.97, staircase=False, name=None)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_rate),loss='sparse_categorical_crossentropy' ,metrics=['acc'],loss_weights={'output':1, 'ax1':0., 'ax2':0})

In [6]:
model_checkpoint_best_path=r'D:\GitHub_repo\CNN-Model-Study\save_weights\GoogleNet/'
model_checkpoint_best=ModelCheckpoint(filepath=model_checkpoint_best_path,monitor='val_acc',save_best_only=True,save_weights_only=True,save_freq='epoch')
early_stopping=EarlyStopping(monitor='val_acc',patience=30,min_delta=1e-4)

In [70]:
history=model.fit(x_train,{'output':y_train,'ax1':y_train,'ax2':y_train},validation_data=(x_test,{'output':y_test,'ax1':y_test,'ax2':y_test}),epochs=10,batch_size=8,verbose=1)

Epoch 1/10
1184/7500 [===>..........................] - ETA: 16:00:09 - loss: 2.3257 - output_loss: 2.3257 - ax1_loss: 2.3293 - ax2_loss: 2.3027 - output_acc: 0.1053 - ax1_acc: 0.1023 - ax2_acc: 0.1019